<a href="https://colab.research.google.com/github/marconebhering/pokedex/blob/main/Pok%C3%A9dex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Importação das Bibliotecas

In [ ]:
import pandas as pd
import requests

## 2. Extração de Dados da API

In [ ]:
# Extrair lista dos primeiros 151 pokemons
def get_pokemons(n_pokemons=151):
  url = 'https://pokeapi.co/api/v2/pokemon/?&limit={}'.format(n_pokemons)
  request = requests.get(url)
  data = request.json()
  results = data['results']
  pokemons = [i['url'] for i in results]
  return pokemons

In [ ]:
# Obter os atributos de cada pokemon
def pokemons():
  pokemons = {'name':[], 'type':[], 'abilities': [], 'hp':[], 'attack':[], 'defense':[], 'speed':[], 'weight':[], 'height':[], "artwork":[]}
  for i in get_pokemons():
    request = requests.get(i)
    data = request.json()
    pokemons['name'].append(data['name'])
    if len(data['types']) == 1:
      pokemons['type'].append(data['types'][0]['type']['name'])
    else:
      pokemons['type'].append(data['types'][0]['type']['name'] + '|' + data['types'][1]['type']['name'])
    if len(data['abilities']) == 1:
      pokemons['abilities'].append(data['abilities'][0]['ability']['name'])
    else:
      pokemons['abilities'].append(data['abilities'][0]['ability']['name'] + '|' + data['abilities'][1]['ability']['name'])
    pokemons['hp'].append(data['stats'][0]['base_stat'])
    pokemons['attack'].append(data['stats'][1]['base_stat'])
    pokemons['defense'].append(data['stats'][2]['base_stat'])
    pokemons['speed'].append(data['stats'][5]['base_stat'])
    pokemons['weight'].append(data['weight'])
    pokemons['height'].append(data['height'])
    pokemons['artwork'].append(data['sprites']['other']['official-artwork']['front_default'])
  return pd.DataFrame.from_dict(pokemons)

In [ ]:
pokemons = pokemons()

In [ ]:
pokemons.head()

,name,type,abilities,hp,attack,defense,speed,weight,height,artwork
0,bulbasaur,grass|poison,overgrow|chlorophyll,45,49,49,45,69,7,https://raw.githubusercontent.com/PokeAPI/spri...
1,ivysaur,grass|poison,overgrow|chlorophyll,60,62,63,60,130,10,https://raw.githubusercontent.com/PokeAPI/spri...
2,venusaur,grass|poison,overgrow|chlorophyll,80,82,83,80,1000,20,https://raw.githubusercontent.com/PokeAPI/spri...
3,charmander,fire,blaze|solar-power,39,52,43,65,85,6,https://raw.githubusercontent.com/PokeAPI/spri...
4,charmeleon,fire,blaze|solar-power,58,64,58,80,190,11,https://raw.githubusercontent.com/PokeAPI/spri...


In [ ]:
# Obter as evoluções de cada pokemon
def get_evolutions():
  evolves = {'name':[], 'evolves_from':[]}
  for i in range(1,152):
    request = requests.get('https://pokeapi.co/api/v2/pokemon-species/{}'.format(i))
    data = request.json()
    evolves['name'].append(data['name'])
    if data['evolves_from_species'] is None:
      evolves['evolves_from'].append(data['evolves_from_species'])
    else:
      evolves['evolves_from'].append(data['evolves_from_species']['name'])
  df = pd.DataFrame.from_dict(evolves)
  merge = df.merge(df, how='left', left_on='name', right_on='evolves_from')
  drop = merge.drop(columns=['evolves_from_y'])
  return drop.rename(columns={'name_x':'name', 'evolves_from_x': 'evolves_from', 'name_y': 'evolves_to'})

In [ ]:
evolutions = get_evolutions()

In [ ]:
# Cabeçalho do dataset de evoluções
evolutions.head()

,name,evolves_from,evolves_to
0,bulbasaur,None,ivysaur
1,ivysaur,bulbasaur,venusaur
2,venusaur,ivysaur,NaN
3,charmander,None,charmeleon
4,charmeleon,charmander,charizard


In [ ]:
# Merge para obter os dados de evolução
def merge_df(pokemons, evolutions):
  merge_df = pokemons.merge(evolutions, how='left', on='name')
  evolves_from = merge_df.merge(pokemons, how='left', left_on='evolves_from', right_on='name')
  evolves_to = evolves_from.merge(pokemons, how='left', left_on='evolves_to', right_on='name')
  evolves_to.drop(columns=['name_y', 'type_y', 'abilities_y', 'hp_y', 'attack_y', 'defense_y', 'speed_y', 'weight_y', 'height_y', 'name',
       'type', 'abilities', 'hp', 'attack', 'defense', 'speed', 'weight',
       'height'], inplace=True)
  evolves_to.rename(columns={'name_x':'name', 'type_x':'type', 'abilities_x':'abilities', 'hp_x':'hp', 'attack_x':'attack', 'defense_x':'defense',
                             'speed_x':'speed', 'weight_x':'weight', 'height_x':'height', 'artwork_x':'artwork', 'artwork_y':'artwork_evolves_from',
                             'artwork':'artwork_evolves_to'}, inplace=True)
  return evolves_to

In [ ]:
df = merge_df(pokemons, evolutions)

## 3. Tratamento dos Dados

In [ ]:
# Quebrar colunas type e abilities em duas novas e capitalizar strings
def poke_transform():
  df[['ability_1', 'ability_2']] = df['abilities'].str.split('|', 1, expand=True)
  df[['type_1', 'type_2']] = df['type'].str.split('|', 1, expand=True)
  df.drop(columns=['abilities', 'type'], inplace=True)
  str_columns = ['name', 'ability_1', 'ability_2', 'type_1', 'type_2', 'evolves_from', 'evolves_to']
  for i in str_columns:
    df[i] = df[i].str.capitalize()
  return df

In [ ]:
df_pokemons = poke_transform()

In [ ]:
df_pokemons.head()

,name,hp,attack,defense,speed,weight,height,artwork,evolves_from,evolves_to,artwork_evolves_from,artwork_evolves_to,ability_1,ability_2,type_1,type_2
0,Bulbasaur,45,49,49,45,69,7,https://raw.githubusercontent.com/PokeAPI/spri...,None,Ivysaur,NaN,https://raw.githubusercontent.com/PokeAPI/spri...,Overgrow,Chlorophyll,Grass,Poison
1,Ivysaur,60,62,63,60,130,10,https://raw.githubusercontent.com/PokeAPI/spri...,Bulbasaur,Venusaur,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,Overgrow,Chlorophyll,Grass,Poison
2,Venusaur,80,82,83,80,1000,20,https://raw.githubusercontent.com/PokeAPI/spri...,Ivysaur,NaN,https://raw.githubusercontent.com/PokeAPI/spri...,NaN,Overgrow,Chlorophyll,Grass,Poison
3,Charmander,39,52,43,65,85,6,https://raw.githubusercontent.com/PokeAPI/spri...,None,Charmeleon,NaN,https://raw.githubusercontent.com/PokeAPI/spri...,Blaze,Solar-power,Fire,None
4,Charmeleon,58,64,58,80,190,11,https://raw.githubusercontent.com/PokeAPI/spri...,Charmander,Charizard,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,Blaze,Solar-power,Fire,None


## 4. Download dos arquivos CSV's

In [ ]:
df_pokemons.to_csv('df_pokemons.csv', index = False)